<a href="https://colab.research.google.com/github/neel26desai/cmpe255_assignment4/blob/main/Apache_Beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-beam

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json
import logging
from urllib.request import urlopen
import argparse
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [25]:
#reading the data
with beam.Pipeline(options=PipelineOptions()) as p:
    rows = p | 'Read from CSV' >> beam.io.ReadFromText('/content/drive/MyDrive/airline_delay.csv', skip_header_lines=1)


INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function annotate_downstream_side_inputs at 0x7ed63e8f3be0> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function fix_side_input_pcoll_coders at 0x7ed63e8f3d00> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function pack_combiners at 0x7ed63e8f8280> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function lift_combiners at 0x7ed63e8f8310> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function expand_sdf at 0x7ed63e8f84c0> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function expand_gbk at 0x7ed63e8f85

In [26]:
def parse_csv(row):
    field_names = ['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
                   'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
                   'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
                   'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
                   'security_delay', 'late_aircraft_delay']
    return dict(zip(field_names, row.split(',')))

def filter_missing_data(row):
    # Assuming empty fields are missing values
    return all(row[field] for field in row)
#removing rows with missing values
cleaned_rows = rows | 'Parse CSV' >> beam.Map(parse_csv) | 'Filter Missing Data' >> beam.Filter(filter_missing_data)


In [27]:
#createing a composite transform for removing the cleaning the data
class CleanData(beam.PTransform):
    def expand(self, pcoll):
        return (pcoll
                | 'Parse CSV' >> beam.Map(parse_csv)
                | 'Filter Missing Data' >> beam.Filter(filter_missing_data))


In [28]:
#using it in pipeline
cleaned_rows = rows | 'Clean Data' >> CleanData()


In [29]:
#usig windowing to group data at 5 minute interval
windowed_rows = (cleaned_rows
                 | 'Window' >> beam.WindowInto(
                     beam.window.FixedWindows(5*60),  # 5 minutes window
                ))


In [30]:
#pardo: log year and month for each valid record
class LogYearMonth(beam.DoFn):
    def process(self, element):
        print(f"Year: {element['year']}, Month: {element['month']}")
        yield element

logged_rows = windowed_rows | 'Log Year and Month' >> beam.ParDo(LogYearMonth())


In [31]:
#saving the processes data as a csv file
windowed_rows | 'Write to Output' >> beam.io.WriteToText('/content/drive/MyDrive/proceesed_file', file_name_suffix=".csv", shard_name_template='')

<PCollection[[31]: Write to Output/Write/WriteImpl/FinalizeWrite.None] at 0x7ed63cb92f80>